In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import random
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM , Dense,GlobalMaxPooling1D,Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

In [2]:
 with open('conversation.json') as content:
    data1 = json.load(content)

In [3]:
#getting all the data to lists
tags = []
inputs = []
responses={}
for intent in data1['intents']:
    responses[intent['tag']]=intent['responses']
    for lines in intent['input']:
        inputs.append(lines)
        tags.append(intent['tag'])
        
#converting to dataframe
data = pd.DataFrame({"inputs":inputs,
                     "tags":tags})

In [4]:
data

,inputs,tags
0,selamat,ucapan_awal
1,halo,ucapan_awal
2,hai,ucapan_awal
3,selamat pagi,ucapan_pagi
4,pagi,ucapan_pagi
5,selamat siang,ucapan_siang
6,siang,ucapan_siang
7,selamat malam,ucapan_malam
8,malam,ucapan_malam
9,siapa kamu,deskripsi_bot


In [5]:
#removing punctuations
import string
data['inputs'] = data['inputs'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['inputs'] = data['inputs'].apply(lambda wrd: ''.join(wrd))
#tokenize the data
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['inputs'])
train = tokenizer.texts_to_sequences(data['inputs'])

#apply padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(train)

#encoding the outputs
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(data['tags'])

In [6]:
 #input length
input_shape = x_train.shape[1]
print(input_shape)
#define vocabulary
vocabulary = len(tokenizer.word_index)
print("number of unique words : ",vocabulary)
#output length
output_length = le.classes_.shape[0]
print("output length: ",output_length)

4
number of unique words :  24
output length:  9


In [7]:
#creating the model
i = Input(shape=(input_shape,))
x = Embedding(vocabulary+1,10)(i)
x = LSTM(10,return_sequences=True)(x)
x = Flatten()(x)
x = Dense(output_length,activation="softmax")(x)
model  = Model(i,x)
#compiling the model
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
#training the model
train = model.fit(x_train,y_train,epochs=200)

Epoch 1/200
1/1 [==============================] - 3s 3s/step - loss: 2.1984 - accuracy: 0.0400
Epoch 2/200
1/1 [==============================] - 0s 11ms/step - loss: 2.1974 - accuracy: 0.0800
Epoch 3/200
1/1 [==============================] - 0s 11ms/step - loss: 2.1964 - accuracy: 0.1600
Epoch 4/200
1/1 [==============================] - 0s 9ms/step - loss: 2.1954 - accuracy: 0.2000
Epoch 5/200
1/1 [==============================] - 0s 9ms/step - loss: 2.1944 - accuracy: 0.2400
Epoch 6/200
1/1 [==============================] - 0s 13ms/step - loss: 2.1934 - accuracy: 0.2800
Epoch 7/200
1/1 [==============================] - 0s 8ms/step - loss: 2.1925 - accuracy: 0.2800
Epoch 8/200
1/1 [==============================] - 0s 12ms/step - loss: 2.1914 - accuracy: 0.2800
Epoch 9/200
1/1 [==============================] - 0s 11ms/step - loss: 2.1904 - accuracy: 0.3200
Epoch 10/200
1/1 [==============================] - 0s 10ms/step - loss: 2.1894 - accuracy: 0.3200
Epoch 11/200
1/1 [======

In [8]:
 while True:
    texts_p = []
    prediction_input = input('You : ')

    # removing punctuation and converting to lowercase
    prediction_input = [letters.lower(
    ) for letters in prediction_input if letters not in string.punctuation]
    prediction_input = ''.join(prediction_input)
    texts_p.append(prediction_input)

    #tokenizing and padding
    prediction_input = tokenizer.texts_to_sequences(texts_p)
    prediction_input = np.array(prediction_input).reshape(-1)
    prediction_input = pad_sequences([prediction_input], input_shape)

    # getting output from model
    output = model.predict(prediction_input)
    output = output.argmax()

    # finding the right tag and predicting
    response_tag = le.inverse_transform([output])[0]
    print("AI : ", random.choice(responses[response_tag]))
    if response_tag == "goodbye":
        break

You :  goodbye


AI :  Terimakasih sampai jumpa di BPS Bondowoso


In [10]:
model = model.save("chatbot.h5")

AttributeError: 'NoneType' object has no attribute 'save'